In [1]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import precision_score, accuracy_score , f1_score , roc_auc_score

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

from mypipelines import *

In [2]:
st_train = pd.read_csv("D:\Main Data\Downloads\store_train.csv",encoding='Latin-1' )
st_test = pd.read_csv("D:\Main Data\Downloads\store_test.csv",encoding='Latin-1')

In [3]:
st_train.transpose()

,0,1,2,3,4,5,6,7,8,9,...,3328,3329,3330,3331,3332,3333,3334,3335,3336,3337
Id,2300919770,5000129575,2501308470,603599999,5002760100,2500937995,2610399999,3718399999,1208999999,505799999,...,1311199999,5513399999,2900799999,2502327795,1904399999,2502354415,4814599999,2122399999,2104599999,2302303355
sales0,848,925,924,924,1017,1494,691,918,931,584,...,641,896,716,1133,593,1133,643,737,558,873
sales1,588,717,616,646,730,1071,476,663,628,455,...,471,579,473,862,420,862,432,507,410,693
sales2,666,780,739,683,735,1196,541,774,775,477,...,474,713,529,867,500,867,475,592,412,736
sales3,1116,1283,1154,1292,1208,1861,861,1189,1228,727,...,806,1143,978,1446,836,1446,887,1020,760,1132
sales4,1133,1550,1314,1297,1326,2023,923,1477,1502,949,...,1135,1236,1106,1531,959,1531,891,1154,782,1425
country,9,1,13,35,27,9,103,183,89,57,...,111,133,7,23,43,23,145,223,45,23
State,23,50,25,6,50,25,26,37,12,5,...,13,55,29,25,19,25,48,21,21,23
CouSub,19770,29575,8470,99999,60100,37995,99999,99999,99999,99999,...,99999,99999,99999,27795,99999,54415,99999,99999,99999,3355
countyname,Hancock County,Addison County,Hampden County,Lassen County,Windsor County,Essex County,Marquette County,Wake County,Nassau County,Hempstead County,...,Fannin County,Waukesha County,Audrain County,Plymouth County,Clayton County,Plymouth County,Falls County,Trimble County,Casey County,Sagadahoc County


In [4]:
st_test.info()
st_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1431 entries, 0 to 1430
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              1431 non-null   int64  
 1   sales0          1431 non-null   int64  
 2   sales1          1431 non-null   int64  
 3   sales2          1431 non-null   int64  
 4   sales3          1431 non-null   int64  
 5   sales4          1431 non-null   int64  
 6   country         1430 non-null   float64
 7   State           1431 non-null   int64  
 8   CouSub          1431 non-null   int64  
 9   countyname      1431 non-null   object 
 10  storecode       1431 non-null   object 
 11  Areaname        1431 non-null   object 
 12  countytownname  1431 non-null   object 
 13  population      1430 non-null   float64
 14  state_alpha     1431 non-null   object 
 15  store_Type      1431 non-null   object 
dtypes: float64(2), int64(8), object(6)
memory usage: 179.0+ KB
<class 'pandas.core

In [5]:
p1 = Pipeline([
('s1',VarSelector(['sales0' ,'sales1', 'sales2', 'sales3', 'sales4', 'country','State'])),  
('s2',DataFrameImputer())
])

p2 = Pipeline([
('s1',VarSelector(['countyname','countytownname' ,'state_alpha','store_Type','population'])),
('s2',DataFrameImputer()),
('s3',get_dummies_Pipe(20))    
])

In [6]:
union_pipe = FeatureUnion([
    ('pipe1',p1),
    ('pipe2',p2)
])

In [7]:
union_pipe.fit(st_train)

FeatureUnion(transformer_list=[('pipe1',
                                Pipeline(steps=[('s1',
                                                 VarSelector(feature_names=['sales0',
                                                                            'sales1',
                                                                            'sales2',
                                                                            'sales3',
                                                                            'sales4',
                                                                            'country',
                                                                            'State'])),
                                                ('s2', DataFrameImputer())])),
                               ('pipe2',
                                Pipeline(steps=[('s1',
                                                 VarSelector(feature_names=['countyname',
                                                                            'countytownname',
                                                                            'state_alpha',
                                                                            'store_Type',
                                                                            'population'])),
                                                ('s2', DataFrameImputer()),
                                                ('s3',
                                                 get_dummies_Pipe(freq_cutoff=20))]))])

In [8]:
x_train = pd.DataFrame(data = union_pipe.fit_transform(st_train),
          columns = union_pipe.get_feature_names_out())

x_train.head()

,pipe1__sales0,pipe1__sales1,pipe1__sales2,pipe1__sales3,pipe1__sales4,pipe1__country,pipe1__State,pipe2__countyname_Washington County,pipe2__countyname_Franklin County,pipe2__countyname_Middlesex County,...,pipe2__state_alpha_SC,pipe2__state_alpha_ND,pipe2__state_alpha_RI,pipe2__state_alpha_WA,pipe2__state_alpha_NM,pipe2__state_alpha_AK,pipe2__state_alpha_UT,pipe2__store_Type_Supermarket Type1,pipe2__store_Type_Grocery Store,pipe2__store_Type_Supermarket Type3
0,848,588,666,1116,1133,9,23,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,925,717,780,1283,1550,1,50,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,924,616,739,1154,1314,13,25,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,924,646,683,1292,1297,35,6,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1017,730,735,1208,1326,27,50,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [9]:
x_test =  pd.DataFrame(data = union_pipe.transform(st_test),
          columns = union_pipe.get_feature_names_out())
x_test.head()

,pipe1__sales0,pipe1__sales1,pipe1__sales2,pipe1__sales3,pipe1__sales4,pipe1__country,pipe1__State,pipe2__countyname_Washington County,pipe2__countyname_Franklin County,pipe2__countyname_Middlesex County,...,pipe2__state_alpha_SC,pipe2__state_alpha_ND,pipe2__state_alpha_RI,pipe2__state_alpha_WA,pipe2__state_alpha_NM,pipe2__state_alpha_AK,pipe2__state_alpha_UT,pipe2__store_Type_Supermarket Type1,pipe2__store_Type_Grocery Store,pipe2__store_Type_Supermarket Type3
0,696.0,511.0,514.0,867.0,1034.0,17.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,599.0,481.0,500.0,883.0,894.0,19.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,599.0,423.0,475.0,802.0,1061.0,21.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,599.0,459.0,462.0,883.0,886.0,35.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,599.0,481.0,505.0,746.0,801.0,37.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
x_train.shape, x_test.shape

((3338, 77), (1431, 77))

In [11]:
sum(st_train['store'] == 0) , sum(st_train['store'] == 1) , 1875 + 1463

(1875, 1463, 3338)

In [12]:
y_train = st_train['store']

y_train

0       0
1       0
2       1
3       0
4       0
       ..
3333    1
3334    0
3335    1
3336    0
3337    1
Name: store, Length: 3338, dtype: int64

In [13]:
y_train.shape
# st_train['store'] == 0   , st_train['store'] == 1

(3338,)

In [14]:
x_train_model, x_validation , y_train_model , y_validation = train_test_split(x_train,y_train, train_size=0.8)

In [15]:
x_train_model.shape, x_validation.shape , y_train_model.shape , y_validation.shape

((2670, 77), (668, 77), (2670,), (668,))

In [16]:
logistic_model_1 = LogisticRegression()

logistic_model_1.fit(x_train_model,y_train_model)

LogisticRegression()

In [17]:
logistic_model_1 = LogisticRegression()

logistic_model_1.fit(x_train,y_train)

LogisticRegression()

In [18]:
logistic_model_1.coef_

array([[-4.64928183e-03, -5.43925675e-03,  1.12870905e-02,
         7.21235394e-04,  1.26510527e-03,  2.61842312e-04,
        -4.64941072e-03, -1.71304571e-01,  1.01599304e-01,
         2.46997509e-01, -2.59201087e-01,  5.50347859e-01,
        -9.15369443e-02,  8.51950723e-02, -2.43115485e-01,
        -2.30019057e-01, -2.68973846e-01, -2.44716119e-01,
         2.48073555e-01, -2.43577131e-01, -1.34435872e-01,
         3.54657207e-01,  2.08762547e-01, -1.89541305e-01,
         2.27535011e-01, -1.26165563e-01,  2.26250352e-01,
        -1.92332004e-01,  2.72838057e-01, -2.40790590e-01,
         8.83852791e-02, -1.10772750e+00,  1.63903227e+00,
        -6.38155755e-01, -1.08518169e+00,  2.39796210e-01,
         6.60929919e-02, -3.95749824e-02,  3.68004316e-01,
        -1.67622889e-01, -1.60889555e-01, -3.09265086e-01,
        -1.34934054e-01, -7.69097780e-02, -3.85303944e-01,
         9.20796773e-02,  1.35489864e-02,  3.35796640e-01,
        -1.52157123e-01, -8.77263118e-02, -3.43644864e-0

In [19]:
y_validation_predict = logistic_model_1.predict(x_validation)

y_validation_predict , y_validation_predict.shape

(array([0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
        0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
        0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
        1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
        1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 

In [20]:
y_validation_predict_prob = logistic_model_1.predict_proba(x_validation)

y_validation_predict_prob , y_validation_predict_prob.shape

(array([[0.53100939, 0.46899061],
        [0.85919745, 0.14080255],
        [0.3321368 , 0.6678632 ],
        ...,
        [0.76595743, 0.23404257],
        [0.14240381, 0.85759619],
        [0.34508135, 0.65491865]]),
 (668, 2))

In [21]:
y_validation_predict_prob[:,1]

array([0.46899061, 0.14080255, 0.6678632 , 0.1885641 , 0.51511675,
       0.91574692, 0.40417114, 0.65051641, 0.25324468, 0.21278423,
       0.84434537, 0.92366412, 0.35410352, 0.44911879, 0.35164125,
       0.21023555, 0.38362297, 0.73567564, 0.21829552, 0.35164125,
       0.11139028, 0.53253962, 0.39617426, 0.19691567, 0.2141274 ,
       0.43258531, 0.41175198, 0.11139028, 0.86992685, 0.59565656,
       0.21176242, 0.11139028, 0.45077505, 0.48141435, 0.15907893,
       0.18814916, 0.20794084, 0.25273658, 0.28247207, 0.98412415,
       0.97724134, 0.37019701, 0.53350253, 0.1983366 , 0.56539851,
       0.23278753, 0.27520657, 0.15574579, 0.43278461, 0.38737274,
       0.45040461, 0.11139028, 0.49572563, 0.18924535, 0.24864328,
       0.21972954, 0.14080255, 0.64793151, 0.320368  , 0.52156354,
       0.11139028, 0.56974473, 0.2355522 , 0.42475109, 0.42170777,
       0.67002467, 0.25842514, 0.28908925, 0.33252467, 0.1676437 ,
       0.16278035, 0.35701   , 0.30618845, 0.72518956, 0.41170

In [22]:
accuracy_score(y_validation ,y_validation_predict)

0.7215568862275449

In [23]:
roc_auc_score(y_validation ,y_validation_predict)

0.7014040986958695

In [24]:
roc_auc_score(y_validation ,y_validation_predict_prob[:,1])

0.773462989048939

In [25]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define hyperparameters
xgb_params = {
    'n_estimators': [10, 20, 30, 40, 50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.4, 0.8, 1],
    'max_depth': [1, 2, 3, 4, 5, 6],
    'subsample': [0.5, 0.8, 1],
    'colsample_bytree': [i/10.0 for i in range(5, 10)],
    'reg_alpha': [1e-5, 1e-2, 0.1, 1, 100],
    'reg_lambda': [1e-5, 1e-2, 0.1, 1, 100]
}

# Perform RandomizedSearchCV
random_xgb_search = RandomizedSearchCV(xgb_model,
                                       param_distributions=xgb_params,
                                       cv=5,
                                       n_iter=10,
                                       scoring='roc_auc',
                                       random_state=42,
                                       n_jobs=-1)

# Fit the model
random_xgb_search.fit(x_train, y_train)

# Get the best model and score
best_model = random_xgb_search.best_estimator_
best_score = random_xgb_search.best_score_

print(f'Best AUC score from RandomizedSearchCV: {best_score}')

# Predict probabilities for the validation set
val_probs = best_model.predict_proba(x_validation)[:, 1]

# Calculate AUC score on the validation set
auc_score_val = roc_auc_score(y_validation, val_probs)
print(f'AUC score on validation set: {auc_score_val}')

# Predict probabilities for the test set
y_test_pred_1 = best_model.predict_proba(x_test)[:, 1]
y_test_pred_1.shape

Best AUC score from RandomizedSearchCV: 0.8118874935714621
AUC score on validation set: 0.9307038669514245


(1431,)

In [26]:
res = pd.DataFrame(y_test_pred_1)
#res.index = x_test.index # its important for comparison
res.columns = ["score"]
open("Property Price Prediction.csv", newline='')
res.to_csv("Property Price Prediction.csv",index = False)